In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [ ]:
fmnist_data = keras.datasets.fashion_mnist

In [ ]:
(x_train, y_train), (x_test, y_test) = fmnist_data.load_data()

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
y_train.shape

In [ ]:
y_train[:10]

In [ ]:
plt.imshow(x_train[0,], cmap="gray")